In [1]:
!pip install jupyter-dash plotly pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.1 MB/s eta 0:00:00


In [25]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from dash import Dash # Changed JupyterDash to Dash
from dash import dcc, html, Input, Output

In [27]:
file = 'https://raw.githubusercontent.com/prince381/marketing_dashboard_design/master/data%20_files/Sample%20-%20Superstore.csv'
file2 = 'https://raw.githubusercontent.com/prince381/marketing_dashboard_design/master/data%20_files/state-abbrevs.csv'

In [32]:
url = "https://raw.githubusercontent.com/prince381/marketing_dashboard_design/master/data%20_files/Sample%20-%20Superstore.csv"
df = pd.read_csv(url)
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [33]:
data = pd.read_csv(file)
data['Category'] = data['Category'].str.strip().str.title()  # remove spaces, standardize
data = data.dropna(subset=['Sales','Profit'])                # drop any blank rows


In [34]:
states = pd.read_csv(file2)
states.columns = ['State','State-abbreviation']
data = pd.read_csv(file)


In [35]:
state_tup = list(zip(states['State'], states['State-abbreviation']))
state_map = dict(state_tup)
data['State-abbreviation'] = data['State'].map(state_map)

In [36]:
columns_to_drop_candidate = ['Row ID','Order ID','Customer ID','Customer Name',
                   'Segment','Country','City','Postal Code','Product ID',
                   'Product Name','Ship Mode']

# Filter columns_to_drop to only include those present in the DataFrame
existing_columns_to_drop = [col for col in columns_to_drop_candidate if col in data.columns]
data = data.drop(existing_columns_to_drop, axis=1)

data['Order Date'] = pd.to_datetime(data['Order Date'])
# Removed: data['Ship Date'] = pd.to_datetime(data['Ship Date'])
data['Year'] = data['Order Date'].dt.year.astype(str)

# Check if 'Ship Date' still exists after the previous drop, before dropping it again
if 'Ship Date' in data.columns:
    data = data.drop('Ship Date', axis=1)

data.State = data.State.str.strip()
market_data = data.copy()

In [37]:
def filter_data(df, category='all', year='all'):
    if category == 'all' and year == 'all':
        return df
    elif category == 'all':
        return df[df.Year == year]
    elif year == 'all':
        return df[df.Category == category]
    return df[(df.Category == category) & (df.Year == year)]

In [38]:
def mkt_summary(df):
    d = df.groupby('Year')[['Sales','Quantity','Profit']].sum().T.reset_index()
    d.columns = ['Metric','2014','2015','2016','2017']
    d['Pct_change'] = np.round(((d['2017'] - d['2016']) / d['2016']) * 100, 2)
    sales = (d.iloc[0,-2:][0], d.iloc[0,-2:][1])
    profit = (d.iloc[2,-2:][0], d.iloc[2,-2:][1])
    quantity = (d.iloc[1,-2:][0], d.iloc[1,-2:][1])
    discount = np.round(df.groupby('Year')['Discount'].mean()['2017'], 2)
    return sales, profit, quantity, discount

In [39]:
def text_prefix_color(num):
    return ('-', 'red') if num < 0 else ('+', 'lightgreen')

In [40]:
def sales_profit_scatter(df):
    d = df[['Order Date','Sales','Profit']].set_index('Order Date').resample('M').sum()
    traces = []
    colors = ['brown','grey']
    for col,color in zip(d.columns,colors):
        traces.append(go.Scatter(x=d.index, y=d[col], name=col, mode='lines',
                                 line={'color':color}))
    layout = go.Layout(height=240, title='Sales vs Profit Over Time')
    return traces, layout

In [41]:
def quantity_scatter(df):
    d = df[['Order Date','Quantity']].set_index('Order Date').resample('M').sum()
    trace = [go.Scatter(x=d.index, y=d['Quantity'], mode='lines', line={'color':'brown'})]
    layout = go.Layout(height=240, title='Quantity Sold Over Time')
    return trace, layout

In [42]:
def quantity_pie(df):
    d = df.groupby('Region')['Quantity'].sum()
    pie_chart = [go.Pie(labels=d.index, values=d.values, hole=.5)]
    layout = go.Layout(height=240, title='Products Purchased by Region')
    return pie_chart, layout

In [43]:
def choro_map(df, metric):
    d = df.groupby(['State','State-abbreviation'])[metric].sum().reset_index()
    ch_map = [go.Choropleth(
        locations=d['State-abbreviation'],
        locationmode='USA-states',
        z=d[metric],
        colorscale='YlOrRd',
        colorbar={'title': metric}
    )]
    layout = go.Layout(height=300, geo={'scope':'usa'}, title=f'{metric} by State')
    return ch_map, layout


In [44]:
app = Dash(__name__) # Changed JupyterDash to Dash
app.title = 'Marketing Dashboard'

app.layout = html.Div([
    html.H2("📊 Marketing Dashboard", style={'textAlign':'center'}),

    html.Div([
        html.Label("Select Product Category:"),
        dcc.RadioItems(
            id='item-type',
            options=[
                {'label':'All','value':'all'},
                {'label':'Furniture','value':'Furniture'},
                {'label':'Office Supplies','value':'Office Supplies'},
                {'label':'Technology','value':'Technology'}
            ],
            value='all',
            labelStyle={'display':'inline-block','marginRight':'10px'}
        )
    ], style={'textAlign':'center'}),

    html.Br(),
    html.Div(id='summary'),
    html.Br(),

    html.Div([
        dcc.Graph(id='usa-map'),
        dcc.Graph(id='sales-profit'),
        dcc.Graph(id='quantity-pie'),
        dcc.Graph(id='quantity-scatter')
    ])
])

In [45]:
@app.callback(Output('summary','children'), [Input('item-type','value')])
def update_summary(item):
    s, p, q, d = mkt_summary(filter_data(market_data, category=item))
    return html.Div([
        html.H4("Summary Metrics", style={'textAlign':'center'}),
        html.P(f"Total Sales: ${s[0]:,.0f}  |  Profit: ${p[0]:,.0f}  |  Quantity Sold: {q[0]:,.0f}  |  Avg. Discount: {d*100:.2f}%",
               style={'textAlign':'center'})
    ])


In [46]:
@app.callback(Output('usa-map','figure'),
              [Input('item-type','value')])
def update_map(item):
    d = filter_data(market_data, category=item, year='2017')
    data, layout = choro_map(d, 'Sales')
    return {'data': data, 'layout': layout}

In [47]:
@app.callback(Output('sales-profit','figure'),
              [Input('item-type','value')])
def update_scatter(item):
    d = filter_data(market_data, category=item)
    data, layout = sales_profit_scatter(d)
    return {'data': data, 'layout': layout}

In [48]:
@app.callback(Output('quantity-pie','figure'),
              [Input('item-type','value')])
def update_pie(item):
    d = filter_data(market_data, category=item, year='2017')
    data, layout = quantity_pie(d)
    return {'data': data, 'layout': layout}

In [49]:
@app.callback(Output('quantity-scatter','figure'),
              [Input('item-type','value')])
def update_quantity(item):
    d = filter_data(market_data, category=item)
    data, layout = quantity_scatter(d)
    return {'data': data, 'layout': layout}

In [51]:
app.run(mode='inline', debug=False)

<IPython.core.display.Javascript object>